In [46]:
from web3 import Web3
import pandas as pd
from decouple import config
from tqdm import tqdm


In [6]:
w3 = Web3(Web3.HTTPProvider(config('MAINNET_ACCESS_URL')))
w3.isConnected()

True

In [28]:
address = Web3.toChecksumAddress("0x0ba45a8b5d5575935b8158a88c631e9f9c95a2e5")
path = os.path.join('..','abi', 'abi.json')
with open(path) as f:
    abi = f.read()
tellor = w3.eth.contract(address=address, abi=abi)

In [37]:
event_filter = w3.eth.filter({
    "startBlock": w3.eth.getBlock('latest').number - 195000,
    "lastBlock": w3.eth.getBlock('latest').number,
    "address": address,
    "topics": ["0x1d85ce10456e29b67de37887496d3f1fcf1b64c79c4d07484038703a9f5c1408"] 
})

In [41]:
event_filter = tellor.events.NewChallenge.createFilter(
    fromBlock=w3.eth.getBlock('latest').number - 195000)

In [43]:
challenges = event_filter.get_all_entries()

In [44]:
len(challenges)

1451

In [49]:
timestamps = []
difficulties = []

for i in tqdm(challenges):
    timestamp = w3.eth.getBlock(i['blockNumber']).timestamp
    difficulty = i['args']['_difficulty']

    timestamps.append(timestamp)
    difficulties.append(difficulty)
    del(timestamp)
    del(difficulty)

100%|██████████| 1451/1451 [04:14<00:00,  5.69it/s]


In [ ]:
df = pd.DataFrame(
    list(zip(tiemstamps, difficulties)),
    columns=['Timestamp', 'Network Difficulty'])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9), frameon=False)

font_path = os.path.join("../extras", "Renogare.ttf")
font_main = fm.FontProperties(fname=font_path, size=64)
font_path = os.path.join("../extras", "Roboto-Medium.ttf")
font_body = fm.FontProperties(fname=font_path, size=24)
fname = os.path.split(font_path)[1]
ax = sns.plot(data=df, x='Timestamp', y='NetworkDifficulty', hue='After')
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Address', fontproperties=font_body, fontsize=16, labelpad=10)
ax.set_ylabel('Balance (TRB)', fontproperties=font_body, fontsize=16)
ax.legend(title='Voted...', loc='center left', fontsize=20, title_fontsize=24, frameon=False, prop=font_body).set_title(title='Voted...', prop=font_body)
ax.set_yscale("log")

ax.set_title('V2.6 Passes Unanimously!', fontproperties=font_main)
# ax.title.set_fontsize(48)
for panel in ['top', 'bottom', 'left', 'right']:
    ax.spines[panel].set_visible(False)